In [7]:
from distutils.filelist import glob_to_re
import random
from skimage.feature import graycomatrix, graycoprops
import numpy as np
from pathlib import Path
import cv2
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, auc, accuracy_score, f1_score
from sklearn import preprocessing
from sklearn.utils import resample, shuffle


In [2]:
import os
os.chdir("..")

In [9]:
chall = "chall2"
glcm_feat = pd.read_feather(f"data/processed/features/glcm_features_{chall}.feather")

In [10]:
X = glcm_feat.iloc[:, 1:len(glcm_feat.columns)-2]
y = glcm_feat["class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)
upsample=False
if upsample and chall=="chall2":
    scc = X_train[y_train == "scc"]
    other_df = X_train[y_train != "scc"]
    scc_upsampled = resample(scc, random_state=42, n_samples=int(len(other_df)/2), replace=True)
    X_train_upsampled = pd.concat([scc_upsampled, other_df]).reset_index(drop=True)
    y_train_upsampled = pd.concat(
        [pd.Series(["scc"]*len(scc_upsampled)), y_train[y_train != "scc"]]).reset_index(drop=True)

    le = LabelEncoder()
    y_train = le.fit_transform(y_train_upsampled)
    y_test = le.transform(y_test)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train_upsampled)
    X_test = scaler.transform(X_test)

else:
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.transform(y_test)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)


In [13]:
svc = SVC(kernel='rbf', probability=True, class_weight='balanced', C=1.0)
svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

print(classification_report(y_test, y_pred))
print('Confusion Matrix\n',confusion_matrix(y_test, y_pred))
print('\nAccuracy: ', accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.63      0.61      0.62       617
           1       0.79      0.61      0.69       801
           2       0.14      0.42      0.21       107

    accuracy                           0.60      1525
   macro avg       0.52      0.55      0.51      1525
weighted avg       0.68      0.60      0.63      1525

Confusion Matrix
 [[376  97 144]
 [184 487 130]
 [ 33  29  45]]

Accuracy:  0.5954098360655737


In [14]:
from xgboost import XGBClassifier

xgb_cl = XGBClassifier()

xgb_cl.fit(X_train, y_train)
y_pred = xgb_cl.predict(X_test)

print(classification_report(y_test, y_pred))
print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))
print('\nAccuracy: ', accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.67      0.69      0.68       617
           1       0.73      0.79      0.76       801
           2       0.31      0.04      0.07       107

    accuracy                           0.70      1525
   macro avg       0.57      0.51      0.50      1525
weighted avg       0.67      0.70      0.68      1525

Confusion Matrix
 [[428 185   4]
 [163 633   5]
 [ 51  52   4]]

Accuracy:  0.6983606557377049
